### Packages

In [ ]:
# pip install webdriver-manager

In [5]:
# pip install selenium
# pip install webdriver-manager

# Packages for webscraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

#Package to handle dataframes
import pandas as pd

#Package for progress bar
from tqdm import tqdm

#### Configure webdriver to use Chrome browser

In [6]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:02<00:00, 2.57MB/s]


### Webscraping function

In [7]:
def UKBB_to_DF(ukbbidlist):
    # Initiate empty lists to store data of interest
    Description=[]
    Category = []
    ValueType=[]
    Nbpart = []
    Sex=[] 
    Mean=[]
    Median=[]
    Stdv=[]

    # 
    for ukbbid in tqdm(ukbbidlist):
        driver = webdriver.Chrome(service=s)
        driver.get("https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=" + str(ukbbid))
    # Search by XPATH (put XPATH between """___""")
        Description.append(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[1]/tbody/tr[1]/td[2]""").text)
        Category.append(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[1]/tbody/tr[2]/td[2]/a[1]""").text)
        ValueType.append(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[2]/tbody/tr[1]/td[5]""").text)
        Nbpart.append(int(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[2]/tbody/tr[1]/td[2]""").text.replace(",", "")))
        Sex.append(driver.find_element(by=By.XPATH, value="""//*[@id="main"]/table[2]/tbody/tr[1]/td[8]""").text)
    # Search by text element (write search query e.g. "Mean = " and remove by replacing string)
        try:
            Mean.append(driver.find_element(By.XPATH, value = ("//*[contains(text(),'Mean = ')]")).text.replace("Mean = ", ""))
        except NoSuchElementException:
            Mean.append("NA")
    
        try:    
            Median.append(driver.find_element(By.XPATH, value = ("//*[contains(text(),'Median')]/following::td")).text)
        except NoSuchElementException:
            Median.append("NA")

        try:
            Stdv.append(driver.find_element(By.XPATH, value = ("//*[contains(text(),'Std.dev = ')]")).text.replace("Std.dev = ", ""))
        except NoSuchElementException:
            Stdv.append("NA")

        driver.close()
    # Write dataframe
    df = pd.DataFrame({'FieldID':ukbbidlist,'Description':Description, "Category":Category,
                       "Value_Type":ValueType, "Nb_Participants": Nbpart,
                       'Sex':Sex,"Mean":Mean, "Median": Median, "SD":Stdv}) 
    
    return(df)

### From TL_metadata file

In [23]:
### Load dataframe
df_metadata = pd.read_csv(r'TL_metadata.csv')
### Get list of FieldIDs
fieldids = list(df_metadata["FieldID"])
fieldids = [x for x in fieldids if str(x) != 'nan'] # Remove NaN
fieldids = [int(i) for i in fieldids] # Convert float to int

In [24]:
### Extract information
df = UKBB_to_DF(fieldids)

100%|██████████| 90/90 [03:13<00:00,  2.15s/it]


In [25]:
df

,FieldID,Description,Category,Value_Type,Nb_Participants,Sex,Mean,Median,SD
0,21003,Age when attended assessment centre,Reception,"Integer, years",502411,Both sexes,57.5481,59,8.44123
1,40008,Age at cancer diagnosis,Cancer register,Continuous,118980,Both sexes,60.389,63,12.4027
2,40007,Age at death,Death register,"Continuous, years",37897,Both sexes,70.1278,71.4,7.46689
3,845,Age completed full time education,Education,"Integer, years",337352,Both sexes,16.7349,16,2.33829
4,2754,Age at first live birth,Female-specific factors,"Integer, years",184998,Females only,25.4356,25,4.62531
...,...,...,...,...,...,...,...,...,...
85,48,Waist circumference,Body size measures,"Continuous, cm",500318,Both sexes,90.1805,90,13.4194
86,1528,Water intake,Diet,"Integer, glasses/day",501532,Both sexes,2.86044,2,2.2286
87,30000,White blood cell (leukocyte) count,Blood count,"Continuous, 10^9 cells/Litre",479256,Both sexes,6.88107,6.65,2.13168
88,21002,Weight,Body size measures,"Continuous, Kg",499731,Both sexes,77.8425,76.2,15.8785


In [29]:
df.to_csv('pheno_list_f.csv', index=False, encoding='utf-8')
df.to_excel('pheno_list_f.xlsx', index=False, encoding='utf-8')